# Newton-Raphson Root Finding Method

## Introduction

The Newton-Raphson method (also known as Newton's method) is one of the most powerful and widely used iterative techniques for finding roots of nonlinear equations. Given a function $f(x)$, we seek values of $x$ such that $f(x) = 0$.

## Theoretical Foundation

### Derivation from Taylor Series

The Newton-Raphson method can be derived from the Taylor series expansion of $f(x)$ about a point $x_n$:

$$f(x) = f(x_n) + f'(x_n)(x - x_n) + \frac{f''(x_n)}{2!}(x - x_n)^2 + \mathcal{O}((x - x_n)^3)$$

Truncating after the linear term and setting $f(x) = 0$:

$$0 \approx f(x_n) + f'(x_n)(x - x_n)$$

Solving for $x$ gives us the next approximation $x_{n+1}$:

$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$

This is the **Newton-Raphson iteration formula**.

### Geometric Interpretation

Geometrically, each iteration finds where the tangent line to the curve $y = f(x)$ at the point $(x_n, f(x_n))$ crosses the $x$-axis. The equation of this tangent line is:

$$y - f(x_n) = f'(x_n)(x - x_n)$$

Setting $y = 0$ and solving for $x$ yields the Newton-Raphson formula.

### Convergence Analysis

The Newton-Raphson method exhibits **quadratic convergence** near simple roots. If $\alpha$ is a root of $f(x)$ and $e_n = x_n - \alpha$ is the error at iteration $n$, then:

$$e_{n+1} \approx \frac{f''(\alpha)}{2f'(\alpha)} e_n^2$$

This means the number of correct digits approximately doubles with each iteration. The convergence rate depends on:

1. **Quality of initial guess**: $x_0$ should be sufficiently close to the root
2. **Non-zero derivative**: $f'(x) \neq 0$ near the root
3. **Continuous second derivative**: For the error analysis to hold

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch

# Set up publication-quality plots
plt.rcParams['figure.figsize'] = [12, 10]
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14

## Implementation

### Core Algorithm

We implement the Newton-Raphson method with convergence tracking and error handling for cases where the derivative vanishes.

In [ ]:
def newton_raphson(f, df, x0, tol=1e-10, max_iter=100):
    """
    Newton-Raphson root finding algorithm.
    
    Parameters
    ----------
    f : callable
        The function whose root we seek
    df : callable
        The derivative of f
    x0 : float
        Initial guess
    tol : float
        Convergence tolerance
    max_iter : int
        Maximum number of iterations
    
    Returns
    -------
    root : float
        Approximate root
    iterations : list
        History of x values
    converged : bool
        Whether the method converged
    """
    iterations = [x0]
    x = x0
    
    for i in range(max_iter):
        fx = f(x)
        dfx = df(x)
        
        # Check for zero derivative
        if abs(dfx) < 1e-15:
            print(f"Warning: Near-zero derivative at iteration {i}")
            return x, iterations, False
        
        # Newton-Raphson update
        x_new = x - fx / dfx
        iterations.append(x_new)
        
        # Check convergence
        if abs(x_new - x) < tol:
            return x_new, iterations, True
        
        x = x_new
    
    return x, iterations, False

## Example 1: Finding Square Roots

To find $\sqrt{a}$, we solve $f(x) = x^2 - a = 0$.

With $f'(x) = 2x$, the iteration becomes:

$$x_{n+1} = x_n - \frac{x_n^2 - a}{2x_n} = \frac{1}{2}\left(x_n + \frac{a}{x_n}\right)$$

This is the famous **Babylonian method** for computing square roots.

In [ ]:
# Find sqrt(2)
a = 2
f_sqrt = lambda x: x**2 - a
df_sqrt = lambda x: 2*x

root, iterations, converged = newton_raphson(f_sqrt, df_sqrt, x0=1.0)

print(f"Finding √{a}:")
print(f"{'Iteration':<12} {'x_n':<20} {'Error':<20}")
print("-" * 52)

exact = np.sqrt(a)
for i, x in enumerate(iterations):
    error = abs(x - exact)
    print(f"{i:<12} {x:<20.15f} {error:<20.2e}")

print(f"\nConverged: {converged}")
print(f"Root found: {root:.15f}")
print(f"Exact value: {exact:.15f}")

## Example 2: Transcendental Equation

Consider finding the root of:

$$f(x) = x - \cos(x) = 0$$

This equation arises in various physics problems. The derivative is:

$$f'(x) = 1 + \sin(x)$$

In [ ]:
# Transcendental equation: x = cos(x)
f_trans = lambda x: x - np.cos(x)
df_trans = lambda x: 1 + np.sin(x)

root_trans, iter_trans, conv_trans = newton_raphson(f_trans, df_trans, x0=0.5)

print("Finding root of x = cos(x):")
print(f"{'Iteration':<12} {'x_n':<20} {'|f(x_n)|':<20}")
print("-" * 52)

for i, x in enumerate(iter_trans):
    print(f"{i:<12} {x:<20.15f} {abs(f_trans(x)):<20.2e}")

print(f"\nRoot found: {root_trans:.15f}")
print(f"Verification: f({root_trans:.10f}) = {f_trans(root_trans):.2e}")

## Visualization of Newton-Raphson Iterations

We now create a comprehensive visualization showing:
1. The geometric interpretation of Newton-Raphson iterations
2. Convergence rate analysis
3. Comparison with different initial guesses
4. Error reduction over iterations

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(14, 12))

# Test function: f(x) = x^3 - 2x - 5
f = lambda x: x**3 - 2*x - 5
df = lambda x: 3*x**2 - 2

# Plot 1: Geometric interpretation
ax1 = fig.add_subplot(2, 2, 1)

x_plot = np.linspace(0, 3, 1000)
y_plot = f(x_plot)

ax1.plot(x_plot, y_plot, 'b-', linewidth=2, label=r'$f(x) = x^3 - 2x - 5$')
ax1.axhline(y=0, color='k', linewidth=0.5)
ax1.axvline(x=0, color='k', linewidth=0.5)

# Newton-Raphson iterations starting from x0 = 3
x0 = 3.0
root, iterations, _ = newton_raphson(f, df, x0, tol=1e-12)

colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(iterations)-1))

for i in range(min(4, len(iterations)-1)):
    xi = iterations[i]
    xi1 = iterations[i+1]
    
    # Plot point on curve
    ax1.plot(xi, f(xi), 'o', color=colors[i], markersize=8)
    
    # Plot tangent line
    x_tan = np.linspace(xi1 - 0.5, xi + 0.5, 100)
    y_tan = f(xi) + df(xi) * (x_tan - xi)
    ax1.plot(x_tan, y_tan, '--', color=colors[i], alpha=0.7, linewidth=1.5)
    
    # Vertical line to x-axis
    ax1.plot([xi, xi], [0, f(xi)], ':', color=colors[i], alpha=0.5)
    
    # Label iteration
    ax1.annotate(f'$x_{i}$', xy=(xi, 0), xytext=(xi, -2),
                 fontsize=10, ha='center')

# Mark the root
ax1.plot(root, 0, 'r*', markersize=15, label=f'Root ≈ {root:.6f}')

ax1.set_xlabel('x')
ax1.set_ylabel('f(x)')
ax1.set_title('Geometric Interpretation of Newton-Raphson Method')
ax1.legend(loc='upper left')
ax1.set_xlim([0, 3.5])
ax1.set_ylim([-10, 20])
ax1.grid(True, alpha=0.3)

# Plot 2: Convergence rate (quadratic convergence)
ax2 = fig.add_subplot(2, 2, 2)

errors = [abs(x - root) for x in iterations]
errors = [e for e in errors if e > 1e-16]  # Remove zero errors for log plot

iterations_num = range(len(errors))
ax2.semilogy(iterations_num, errors, 'bo-', linewidth=2, markersize=8, label='Actual error')

# Theoretical quadratic convergence
if len(errors) > 1:
    # Estimate convergence constant
    theoretical = [errors[0]]
    C = abs(df(root) / (2 * (3*root**2 - 2)))  # Approximate constant
    for i in range(1, len(errors)):
        theoretical.append(C * theoretical[-1]**2)
    ax2.semilogy(range(len(theoretical)), theoretical, 'r--', linewidth=1.5, 
                 alpha=0.7, label='Quadratic convergence')

ax2.set_xlabel('Iteration')
ax2.set_ylabel('Error (log scale)')
ax2.set_title('Convergence Rate Analysis')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Different initial guesses
ax3 = fig.add_subplot(2, 2, 3)

initial_guesses = [1.0, 2.0, 3.0, 4.0, 5.0]
colors_init = plt.cm.tab10(np.linspace(0, 1, len(initial_guesses)))

for x0, color in zip(initial_guesses, colors_init):
    _, iters, conv = newton_raphson(f, df, x0, tol=1e-12)
    errs = [abs(x - root) for x in iters]
    errs = [e if e > 1e-16 else 1e-16 for e in errs]
    ax3.semilogy(range(len(errs)), errs, 'o-', color=color, 
                 linewidth=1.5, markersize=6, label=f'$x_0 = {x0}$')

ax3.set_xlabel('Iteration')
ax3.set_ylabel('Error (log scale)')
ax3.set_title('Effect of Initial Guess on Convergence')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Basin of attraction / Function landscape
ax4 = fig.add_subplot(2, 2, 4)

# For a polynomial with multiple roots: f(x) = x^3 - x
f_multi = lambda x: x**3 - x
df_multi = lambda x: 3*x**2 - 1

x_basin = np.linspace(-2, 2, 500)
roots_found = []

for x0 in x_basin:
    try:
        r, _, _ = newton_raphson(f_multi, df_multi, x0, tol=1e-10, max_iter=50)
        roots_found.append(r)
    except:
        roots_found.append(np.nan)

roots_found = np.array(roots_found)

# Color by which root is found
colors_basin = np.zeros(len(x_basin))
colors_basin[np.abs(roots_found - (-1)) < 0.1] = -1
colors_basin[np.abs(roots_found - 0) < 0.1] = 0
colors_basin[np.abs(roots_found - 1) < 0.1] = 1

ax4.scatter(x_basin, roots_found, c=colors_basin, cmap='coolwarm', s=1, alpha=0.7)
ax4.axhline(y=-1, color='b', linestyle='--', alpha=0.5, label='Root: -1')
ax4.axhline(y=0, color='gray', linestyle='--', alpha=0.5, label='Root: 0')
ax4.axhline(y=1, color='r', linestyle='--', alpha=0.5, label='Root: 1')

ax4.set_xlabel('Initial guess $x_0$')
ax4.set_ylabel('Root found')
ax4.set_title(r'Basin of Attraction for $f(x) = x^3 - x$')
ax4.legend(loc='upper left')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## Analysis of Results

### Key Observations

1. **Quadratic Convergence**: The error plot demonstrates that the number of correct digits approximately doubles with each iteration, characteristic of quadratic convergence.

2. **Sensitivity to Initial Guess**: While the method converges from various starting points, the number of iterations required depends on the quality of the initial guess.

3. **Basin of Attraction**: For functions with multiple roots, the initial guess determines which root is found. The basin of attraction can have complex, fractal-like boundaries.

### Limitations and Failure Modes

The Newton-Raphson method can fail when:

- $f'(x_n) = 0$ (horizontal tangent)
- The iterates oscillate or diverge
- The function has discontinuities or is not differentiable

### Extensions

- **Modified Newton's Method**: Uses $x_{n+1} = x_n - m\frac{f(x_n)}{f'(x_n)}$ for roots of multiplicity $m$
- **Quasi-Newton Methods**: Approximate the derivative numerically
- **Multivariate Newton's Method**: Extends to systems of equations using the Jacobian matrix

In [ ]:
# Demonstrate failure mode: cycling behavior
print("Demonstration of potential failure modes:")
print("="*50)

# Function where Newton-Raphson can cycle
f_cycle = lambda x: x**3 - 2*x + 2
df_cycle = lambda x: 3*x**2 - 2

# This starting point leads to slow convergence
x0_bad = 0.0
try:
    root_bad, iter_bad, conv_bad = newton_raphson(f_cycle, df_cycle, x0_bad, max_iter=10)
    print(f"\nStarting from x0 = {x0_bad} for f(x) = x³ - 2x + 2:")
    for i, x in enumerate(iter_bad[:6]):
        print(f"  x_{i} = {x:.10f}")
    print(f"  Converged: {conv_bad}")
except Exception as e:
    print(f"  Failed: {e}")

# Good starting point
x0_good = -2.0
root_good, iter_good, conv_good = newton_raphson(f_cycle, df_cycle, x0_good)
print(f"\nStarting from x0 = {x0_good}:")
for i, x in enumerate(iter_good):
    print(f"  x_{i} = {x:.10f}")
print(f"  Converged: {conv_good}")
print(f"  Root: {root_good:.15f}")

## Conclusion

The Newton-Raphson method is a powerful root-finding algorithm characterized by:

- **Quadratic convergence** near simple roots
- **Elegant geometric interpretation** via tangent lines
- **Wide applicability** to differentiable functions

However, practitioners must be aware of its limitations:

- Requires computation of the derivative
- Sensitive to the initial guess
- May fail near stationary points

For production use, the method is often combined with safeguards such as bracketing or line search to ensure global convergence.